In [1]:
import requests
import pandas as pd
import numpy
from bs4 import BeautifulSoup
import datetime as dt
import locale

In [2]:
#locale.setlocale(locale.LC_TIME, "sv_SE.UTF-8") #Unix/Linux?
locale.setlocale(locale.LC_TIME, 'swedish')

'Swedish_Sweden.1252'

In [3]:
url = "https://se.trustpilot.com/review/anicura.se?page={1}"

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
first_page = 1
to_page = 71

In [7]:
titles = []
origin_dates = []
dates = []
ratings = []
review_texts = []
pages = []

In [8]:
replacements = {
    "jan": "Januari",
    "feb": "Februari",
    "apr": "April",
    #"mar": "Mars",
    #"maj": "Maj",
    #"jun": "Juni",
    #"jul": "Juli",
    "aug": "Augusti",
    "sep": "September",
    "okt": "Oktober",
    "nov": "November",
    "dec": "December",  
}

In [9]:
def full_origin_date_text(origin_date):
    origin_date = origin_date.lower()
    origin_date = origin_date.replace('upplevelsen ägde rum: ', "")
    origin_date = origin_date.replace("uppdaterad ", "")
    origin_date = origin_date.replace(".", "").lower()
    for abb, full in replacements.items():
        origin_date = origin_date.replace(abb, full)
    return origin_date

In [10]:
def full_date_text(date):
    date = date.lower()
    date = date.replace('upplevelsen ägde rum: ', "")
    date = date.replace(".", "").lower()
    return date

In [11]:
def date_to_datetime_object(date):
    if "timmar sedan" in date.lower() or "timme sedan" in date.lower():
        date = dt.datetime.now().date()
    elif "en dag sedan" in date.lower():
        date = dt.datetime.now().date() - dt.timedelta(days=1)
    elif "dagar sedan" in date.lower():
        date = dt.datetime.now().date() - dt.timedelta(days=int(date[0]))
    else:
        date = dt.datetime.strptime(date, "%d %B %Y").date()
        
    return date

In [12]:
for i in range(first_page, to_page +1):
    response = requests.get(f"https://se.trustpilot.com/review/anicura.se?page={i}")
    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    
    for review in soup.find_all(class_ = "paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl"):
        #titles
        title = review.find(class_ = "link_internal__7XN06 typography_appearance-default__AAY17 typography_color-inherit__TlgPO link_link__IZzHN link_notUnderlined__szqki")
        titles.append(title.getText())
        
        #if we want one or both dates
        origin_date = review.select_one(selector="time")
        origin_date = full_origin_date_text(origin_date.getText())
        origin_date = date_to_datetime_object(origin_date)
        origin_dates.append(origin_date)
        
        #date text to datetime objects
        date = review.find(class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17").getText().replace("uppdaterad ", "")
        date = full_date_text(date)
        date = date_to_datetime_object(date)
        dates.append(date)
        
        #ratings
        rating = review.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild()
        ratings.append(rating["alt"])
        
        # text or no text
        text = review.find(class_ = "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
        if text == None:
            review_texts.append("")
        else:
            review_texts.append(text.getText())
            
        #pagenumber
        pages.append(i)
        

In [13]:
df_reviews = pd.DataFrame(list(zip(titles, origin_dates,dates, ratings, review_texts, pages)),
                         columns =["Title", "Review Date","Date", "Rating", "Text", "Page"])

In [14]:
df_reviews

,Title,Review Date,Date,Rating,Text,Page
0,5 stjärnor för AniCura Högsby,2024-03-15,2024-03-15,Betygsatt 5 av 5 stjärnor,AniCura i Högsby är en underbar veterinärklini...,1
1,Ni är helt suveräna,2024-03-07,2024-03-06,Betygsatt 5 av 5 stjärnor,Ni är helt suveräna. Ringde i panik i tisdagse...,1
2,"""Vill du boka tid? Vi ringer upp sen. På hemli...",2024-03-11,2024-03-06,Betygsatt 1 av 5 stjärnor,Nu ska det sitta en mötesbokare och ringa upp ...,1
3,Engagerad personal,2023-09-29,2023-09-25,Betygsatt 5 av 5 stjärnor,Engagerad personal! För två år sedan saknade e...,1
4,"Tryggt, snabbt och proffsigt!",2023-09-25,2023-09-24,Betygsatt 5 av 5 stjärnor,Mycket värt att slippa bli runtskickad i telef...,1
...,...,...,...,...,...,...
1414,otippat dåligt,2019-02-20,2019-02-20,Betygsatt 1 av 5 stjärnor,Så fin personal men företaget verkar ha löpt a...,71
1415,Ockrare,2018-09-10,2018-09-10,Betygsatt 1 av 5 stjärnor,VI fick betala nästan 4000kr för att titta til...,71
1416,Missnöjd,2018-12-29,2018-09-08,Betygsatt 2 av 5 stjärnor,Katten fick operera tassen. Skulle byta bandag...,71
1417,Oseriösa,2018-03-02,2018-03-02,Betygsatt 1 av 5 stjärnor,"Oseriösa, röriga, lögnare och väldigt okunniga...",71


In [19]:
#df_reviews.to_csv('TrustPilot_Anicuira.csv', index=False, encoding='utf-8')